In [1]:
from matplotlib import rc
rc("font",**{"family":"serif","serif":["Times"]})
rc("text", usetex=True)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import sncosmo
from tqdm import tqdm
from matplotlib.ticker import ScalarFormatter
from matplotlib.backends.backend_pdf import PdfPages
from scipy.interpolate import interp1d
from scipy import signal
import pickle
import bz2

In [3]:
plt.rcParams["font.size"] = 22.

In [4]:
w_grid=np.arange(3305.,8590.,5.)    # wavelength grid. NEW VALUES!

In [5]:
len(w_grid)

1057

In [6]:
class ScalarFormatterClass1(ScalarFormatter):
    def _set_format(self):
        self.format = "%1.1f"

## Retrieving the spectras

In [7]:
file_names=glob.glob("./data/spectras/*.dat")   # unit: W/m^2/nm times offset, equivalently 10^2 erg/s/cm^2/Angstrom times offset

In [8]:
catalog = pd.read_csv("./data/list.dat", names=["name", "phase", "path"], sep="\\s+")

In [9]:
len(catalog)

2474

In [10]:
sne_list=catalog["name"].unique() # 171 objects

In [11]:
len(catalog["name"].value_counts(sort=True).loc[lambda x : x<=15])

100

In [12]:
len(catalog["name"].value_counts(sort=True))

171

In [13]:
# max: 32 spectras
# min: 5 spectras
# 21% with less or equal 10
# 58% with less or equal 15

We save the figures with the superposition of actual spectra and filtered spectra. And also save 3d figures of the disposition of said spectras in time.

In [14]:
rmv_specs={"Test_SN2":[25.853,27.738,32.374],
           "Train_SN5":[20.635],
           "Test_SN12":[45.809],
           "Train_SN59":[37.662]}

In [15]:
interp_data={}

In [16]:
for sn in tqdm(sne_list):
    spec_list=catalog[catalog["name"]==sn].reset_index(drop=True)
    interp_data[sn]={}
    spec_list.sort_values(by="phase",inplace=True,ignore_index=True)
    
    for i in range(len(spec_list)):
        spec_data=sncosmo.read_lc("./data/spectras/"+spec_list["path"][i], format="salt2")
               
        if (sn in rmv_specs.keys()) and (spec_list["phase"][i] in rmv_specs[sn]):
            continue
        
        # interpolating before appplying the filter
        
        w_min=np.ceil(np.min(spec_data["WAVE"]))
        w_max=np.floor(np.max(spec_data["WAVE"]))
        
        w_grid_eff=w_grid[(w_grid>=w_min) & (w_grid<=w_max)]

        fluxinterp=interp1d(spec_data["WAVE"],spec_data["SN_SPEC"])(w_grid_eff)
        einterp=interp1d(spec_data["WAVE"],spec_data["SN_ERR"])(w_grid_eff)
       
        interp_data[sn][spec_list["phase"][i]]=pd.DataFrame({"wave": w_grid_eff, "flux": fluxinterp, "eflux": einterp})

100%|█████████████████████████████████████████| 171/171 [00:03<00:00, 50.92it/s]


In [17]:
for sn in tqdm(sne_list):
    pp = PdfPages(f"./plots_interpolated_spectras/{sn}_v2.pdf")
    
    for i in interp_data[sn].keys():    

        fig,ax=plt.subplots(figsize=(10,8))       

        ax.plot(interp_data[sn][i]["wave"],interp_data[sn][i]["flux"],label="Spectra at "+str(i)+" days",lw=2,c="C0",zorder=1)
        ax.fill_between(interp_data[sn][i]["wave"],interp_data[sn][i]["flux"]+2*interp_data[sn][i]["eflux"],interp_data[sn][i]["flux"]-2*interp_data[sn][i]["eflux"],color="C0",alpha=0.7,zorder=2,label=r"$95\%$ confidence")
        ax.set_ylabel(r"Flux (erg/s/cm$^2/\AA$ $\times$ offset)")
        ax.set_xlabel(r"Wavelength ($\AA$)")
        ax.legend(fontsize=22)                                   
        ax.autoscale()
        formatter11=ScalarFormatterClass1()
        formatter11.set_scientific(True)
        formatter11.set_powerlimits((0,0))
        ax.yaxis.set_major_formatter(formatter11)
                
        pp.savefig()
        plt.close(fig)
        
    pp.close()

100%|█████████████████████████████████████████| 171/171 [01:59<00:00,  1.43it/s]


In [18]:
for sn in tqdm(sne_list):
    pp = PdfPages(f"./plots_3d_interpolated_spectras/{sn}_v2.pdf")

    fig = plt.figure(figsize=(15,12))
    ax = plt.axes(projection="3d")
    
    for p in interp_data[sn].keys():
        X, Y = np.meshgrid(p,interp_data[sn][p]["wave"].values)
        Z=interp_data[sn][p]["flux"].values.reshape(Y.shape)
        ax.plot_wireframe(X,Y,Z,color="black",lw=2)
        ax.set_xlabel(r"Phase (days)")
        ax.set_ylabel(r"Wavelength ($\AA$)")
        ax.set_zlabel(r"Flux (erg/s/cm$^2/\AA$ $\times$ offset)")
        ax.yaxis.labelpad=20
        ax.xaxis.labelpad=15
        ax.zaxis.labelpad=15
    #plt.show()
    pp.savefig()
    plt.close(fig)
    pp.close()

100%|█████████████████████████████████████████| 171/171 [00:25<00:00,  6.78it/s]


## Saving interpolated spectras

In [19]:
for sn in tqdm(sne_list):   

    f=bz2.BZ2File(f"./interpolated_spectras/{sn}_v2.pkl", "wb")
    pickle.dump(interp_data[sn], f)
    f.close()

100%|█████████████████████████████████████████| 171/171 [00:03<00:00, 44.34it/s]
